In [2]:
import numpy

In [7]:
def testfunc(a,b=2,**cd):
    print(a)
    print('b is',b)
    takecd(**cd)
def takecd(c=3,d=5):
    print(f'c is {c} and d is {d}')

In [8]:
testfunc(1,b=3,c=4,d=5)

1
b is 3
c is 4 and d is 5


In [9]:
testfunc(1,c=4,d=5)

1
b is 2
c is 4 and d is 5
